In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(params):
    """The quantum circuit that you will differentiate!

    Args:
        params (list(float)): The parameters for gates in the circuit
    
    Returns:
        (numpy.array): An expectation value. 
    """
    qml.broadcast(qml.Hadamard, wires=range(3), pattern="single")
    qml.CRX(params[0], [1, 2])
    qml.CRY(params[1], [0, 1])
    qml.CRZ(params[2], [2, 0])
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1) + qml.PauliX(2))

In [3]:
def shifts_and_coeffs():
    s_1 = (np.pi/2)
    s_2 = 3*(np.pi/2)
    c_1 = (np.sqrt(2)+1)/(4*np.sqrt(2))
    c_2 = (np.sqrt(2)-1)/(4*np.sqrt(2))
    
    shifts = [s_1, s_2]
    coeffs = [c_1, c_2]
    return (shifts,coeffs)

def my_parameter_shift_grad(params):

    gradient = np.zeros_like(params)
    shifts, coeffs = shifts_and_coeffs()

    for i in range(len(params)):
        #first we need to make two copies of the parameter list 
        param_first = np.zeros_like(params)
        param_second = np.zeros_like(params)
        param_third = np.zeros_like(params)
        param_fourth = np.zeros_like(params)
        #now we have to change their values in order to make the (Theta+s) (Theta-s)
        for k in  range(len(params)):
            if k == i:
                param_first[k] = (params[k] + shifts[0])
                param_second[k] = (params[k] - shifts[0])
                param_third[k] = (params[k] + shifts[1])
                param_fourth[k] = (params[k] - shifts[1])
            
            else:
                param_first[k] = params[k] 
                param_second[k] = params[k]
                param_third[k] = params[k]
                param_fourth[k] = params[k]
            
        gradient[i] = coeffs[0]*(circuit(param_first) - circuit(param_second)) - coeffs[1]*(circuit(param_third) - circuit(param_fourth))
        
    return np.round_(gradient, decimals=5).tolist()

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    params = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"

In [5]:
test_cases = [['[1.23, 0.6, 4.56]', '[0.08144, -0.33706, -0.37944]']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[1.23, 0.6, 4.56]'...
Correct!
